In [ ]:
from dotenv import load_dotenv
import os
from elasticsearch import helpers  # For bulk Data Uploading
from elasticsearch import Elasticsearch  # Base function for interacting with Elasticsearch
from elasticsearch import RequestError
from pprint import pprint
from load_ini import *

## Connect to Elastic search 
This is done using your own API key, generated using kibana, which should be stored localy on your machine. We stored our API key in a .env file in the same folder as this script, and use the python module dotenv to load it as an environment variable.

In [ ]:
load_dotenv()
client = Elasticsearch("https://localhost:9200/", api_key=os.getenv('API_KEY'),verify_certs=False)

#test client
print(client.info())

## Create the dictionnaries containing all the mappings

We read the mappings list from the metadata.csv file, which is generated externally from the shared excel file, and the list is then transformed into a dictionnary of mappings with the proper formatting.

In [ ]:
# grab mappings list from the csv file, which should be in the same directory as the script
DIR_NAME = os.getcwd()
FILE_NAME = "metadata.csv"
FILE_PATH = os.path.join(DIR_NAME, FILE_NAME)
data, header = read_csv(FILE_PATH)

# create the dictionnary of mappings
mappings = create_mapping(data, header)
pprint(mappings)

Add other settings to the index parameters

In [ ]:
"""
"settings" is technically not needed if we are working on a simple local host, but can be changed to optimise search performance on a database that is hosted on a cluster and searched by multiple users.
"mappings" is required if you wish to explicitly map fields to specific values
"""

index_definition = {
    "settings": {
        "number_of_shards": 1,
    },
    "mappings": {"properties": mappings},
}

Create the index using the parameters set above.

In [ ]:
""" For now we use a test index, but the name of the index should be changed later on"""

try:
    client.indices.create(index="test_index", body=index_definition)
except RequestError:
    print("Index already exists, delete it if you want to recreate it")


## Kibana

Once the index is created in elasticsearch, in order to view it in Kibana, go to **Management** and on left menu bar, scroll down to the **Kibana** subsection, and click **Data Views**. From here, on the top right, click **Create data view** in order to integrate the new index into the Kibana interface. This will allow you to view how kibana interprets the index you have created. 

## Loading data
Now that the index is created and visible in Kibana, we can start mapping models to the index.

In [ ]:
# Define directories and file names here
DIR = "/Users/camille/Documents/runs/phantom/wind"  # Careful, this is a local path - change it to your own
PREFIX = "wind"
MODELS = ["v10e00", "v20e00", "v05e00"]

### Load information from .setup and .in file
We can map the parameters of interest by reading them from our data files, for instance .setup and .in files.



In [ ]:
# Add 1 model
testData = LoadSetupData(os.path.join(DIR,MODELS[0]),PREFIX,index_definition)
pprint(testData)
client.index(index='test_index', body=testData)
del testData

We can also load multiple models at a time, which is preferable of course.

In [ ]:
# Add multiple models
base_command = {"_index": "test_index", "_op_type": "index"}
operations = []
for model in MODELS:
    modelData = LoadSetupData(os.path.join(DIR, model), PREFIX, index_definition)
    operations.append((base_command | {"_source": modelData}))
    del modelData
print(operations)
helpers.bulk(client, operations, refresh=True)
del operations

### Now look into Dashboards: Analytics - Dashboards to visualise data
Documentation: https://www.elastic.co/guide/en/kibana/current/create-a-dashboard-of-panels-with-web-server-data.html

